# Проект для «Викишоп» с BERT

### Содержание:

* [Загрузка данных](#chapter1)
    * [Вывод](#section_1_1)
* [Использование предобученной модели для английского языка DistilBERT](#chapter2)
    * [Логистическая регрессия](#section_2_1)
    * [LightGBM](#section_2_2)
    * [Случайный лес](#section_2_3)
    * [Вывод](#section_2_4)
* [Применение метода TF-IDF](#chapter3)
    * [Логистическая регрессия](#section_3_1)
    * [LightGBM](#section_3_2)
    * [Случайный лес](#section_3_3)
    * [Вывод](#section_3_4)
* [Тестирование лучшей модели](#chapter4) 
* [Итоговый вывод](#chapter5) 

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
!pip install transformers

In [2]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from tqdm import notebook
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem import WordNetLemmatizer 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_extraction.text import TfidfVectorizer

from lightgbm import LGBMClassifier


nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lapsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lapsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Загрузка данных <a class="anchor" id="chapter1"></a>

In [4]:
data = pd.read_csv(r'C:\Users\lapsh\yandex-praktikum-projects\toxic_comments.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

#### Вывод <a class="anchor" id="section_1_1"></a>
Всего в базе данных 159571 записей. Текст комментариев - английский. Отрицательные комментарии составляют 10.2%.

### Использование предобученной модели для английского языка DistilBERT <a class="anchor" id="chapter2"></a>

Для решения поставленной задачи попробуем применить предобученную модель для английского языка. Использовать будем DistilBERT. На ней мы выполним токенизацию текстов и создадим эмбеддингию. Полученные эмбеддинги будем использовать в других моделях для непосредственно классификации.

Из-за органиченных ресурсов, мы не сможем использовать весь набор данных. Ограничимся 10 тыс. записей.

In [8]:
data_1 = data.sample((10000), random_state=12345).reset_index(drop=True) 

In [9]:
data_1['toxic'].value_counts()

0    8960
1    1040
Name: toxic, dtype: int64

Структура отобранных данных сопоставима с исходными. Отрицательные комментарии составляют 10.4%.

In [10]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


In [11]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# токинезация
tokenized = data_1['text'].apply((lambda x: tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True)))

In [13]:
# заполним 0 векторы, длина которых меньше максимальной
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [14]:
np.array(padded).shape

(10000, 512)

In [15]:
# маска важных токенов
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10000, 512)

Начнём преобразование текстов в эмбеддинги. Используем библиотеку tqdm для отображения индикатора прогресса. В Jupyter применим функцию notebook() из этой библиотеки. 
Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча небольшим: 100

In [16]:
batch_size = 100 
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy()) 


  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
# соберём все эмбеддинги в матрицу признаков вызовом функции concatenate():
features = np.concatenate(embeddings)

In [18]:
target = data_1['toxic']

In [19]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.15, stratify=target)

In [20]:
features_train.shape

(8500, 768)

In [21]:
target_train.shape

(8500,)

In [22]:
features_test.shape

(1500, 768)

In [23]:
target_test.shape

(1500,)

Рассмотрим 3 модели: логистическая регрессия, LightGBM, случайный лес.

#### Логистическая регрессия <a class="anchor" id="section_2_1"></a>

In [24]:
model_log = LogisticRegression(random_state=12345, class_weight='balanced')

In [25]:
params_log = {
    'solver': ['liblinear', 'sag','saga','newton-cg'],
    'C': [0.5,1.0,1.5],
    'intercept_scaling':[0.5,1.0,1.5]
}

In [26]:
model_log_grid = RandomizedSearchCV(model_log, params_log, scoring='f1')
model_log_grid.fit(features_train, target_train)

RandomizedSearchCV(estimator=LogisticRegression(class_weight='balanced',
                                                random_state=12345),
                   param_distributions={'C': [0.5, 1.0, 1.5],
                                        'intercept_scaling': [0.5, 1.0, 1.5],
                                        'solver': ['liblinear', 'sag', 'saga',
                                                   'newton-cg']},
                   scoring='f1')

In [27]:
print('Логистическая регрессия. Гиперпараметры ', model_log_grid.best_params_)
print('Логистическая регрессия. F1 ', model_log_grid.best_score_)

Логистическая регрессия. Гиперпараметры  {'solver': 'liblinear', 'intercept_scaling': 1.0, 'C': 1.0}
Логистическая регрессия. F1  0.6649972047874526


#### LightGBM <a class="anchor" id="section_2_2"></a>

In [28]:
model_lgb = LGBMClassifier(random_state=12345, class_weight='balanced')

In [29]:
lgb_params = {
    'objective': ['regression','binary'],
    'boosting_type' : ['dart','gbdt','goss'],
    'max_depth': [15,30],
    'num_leaves': [10,200,250]
}


In [30]:
model_lgb_grid = RandomizedSearchCV(model_lgb, lgb_params, scoring='f1')
model_lgb_grid.fit(features_train, target_train)

RandomizedSearchCV(estimator=LGBMClassifier(class_weight='balanced',
                                            random_state=12345),
                   param_distributions={'boosting_type': ['dart', 'gbdt',
                                                          'goss'],
                                        'max_depth': [15, 30],
                                        'num_leaves': [10, 200, 250],
                                        'objective': ['regression', 'binary']},
                   scoring='f1')

In [31]:
print('LightGBM. Гиперпараметры ', model_lgb_grid.best_params_)
print('LightGBM. F1 ', model_lgb_grid.best_score_)

LightGBM. Гиперпараметры  {'objective': 'binary', 'num_leaves': 250, 'max_depth': 15, 'boosting_type': 'goss'}
LightGBM. F1  0.6693750278114453


#### Случайный лес <a class="anchor" id="section_2_3"></a>

In [32]:
model_forest = RandomForestClassifier(random_state=12345, class_weight='balanced')

In [33]:
forest_params = {
    'max_depth': range(1, 26),
    'n_estimators':range(1, 11)
}

In [34]:
model_forest_grid = RandomizedSearchCV(model_forest, forest_params, scoring='f1')
model_forest_grid.fit(features_train, target_train)

RandomizedSearchCV(estimator=RandomForestClassifier(class_weight='balanced',
                                                    random_state=12345),
                   param_distributions={'max_depth': range(1, 26),
                                        'n_estimators': range(1, 11)},
                   scoring='f1')

In [35]:
print('Случайный лес. Гиперпараметры ', model_forest_grid.best_params_)
print('Случайный лес. F1 ', model_forest_grid.best_score_)

Случайный лес. Гиперпараметры  {'n_estimators': 9, 'max_depth': 9}
Случайный лес. F1  0.608670216909341


#### Вывод <a class="anchor" id="section_2_4"></a>
Ни одна из расмотренных моделей не достигла требуемого значения метрики качества F1 не меньше 0.75.

### Применение метода  TF-IDF<a class="anchor" id="chapter3"></a>

Применим метод TF-IDF:  Проведём лемматизацию слов с помощью WordNetLemmatizer() из библиотеки nltk. Почистим данные от числовых значений, никнеймов и хэштегов.

Почистим данные от числовых значений, никнеймов и хэштегов. Слова с апострофами нормализуем. Для нормализации используем готовый словарь сокращений английских слов.

In [37]:
contractions_dict = { "ain't": "are not", "'s":" is", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "‘cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am", "I've": "I have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "that'd": "that would", "that'd've": "that would have", "there'd": "there would", "there'd've": "there would have", "they'd": "they would", "they'd've": "they would have","they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not","what'll": "what will", "what'll've": "what will have", "what're": "what are", "what've": "what have", "when've": "when have", "where'd": "where did", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who've": "who have", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [38]:
contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    def replace(match):
        return contractions_dict[match.group(0)]
    text = text.lower()
    text = contractions_re.sub(replace, text)
    clean_text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|([^'A-Za-z])|(\w+:\/\/\S+)"," ",text).split())
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, 'v') for w in nltk.word_tokenize(clean_text)]).replace("'", "").strip()

    return lemmatized_output

In [39]:
data['lemmas'] = data['text'].apply(lemmatize)

In [40]:
data.head()

,text,toxic,lemmas
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,daww he match this background colour i m seemi...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it be j...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can not make any real suggestions on im...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Так же, как и в предыдущем блоке, рассмотрим 3 модели: логистическая регрессия, LightGBM, случайный лес.

In [41]:
features_2 = data.drop(['toxic'], axis = 1)
target_2 = data['toxic']

In [42]:
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(features_2, target_2, test_size = 0.2, random_state = 12345, stratify = target_2)

In [43]:
features_2_valid, features_2_test, target_2_valid, target_2_test  = train_test_split(features_2_valid, target_2_valid, test_size = 0.5, random_state = 12345, stratify = target_2_valid)

In [44]:
stopwords = set(nltk_stopwords.words('english'))

In [45]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

In [46]:
features_2_train = count_tf_idf.fit_transform(features_2_train['lemmas'].values.astype('U'))
features_2_valid = count_tf_idf.transform(features_2_valid['lemmas'].values.astype('U'))
features_2_test = count_tf_idf.transform(features_2_test['lemmas'].values.astype('U'))

In [47]:
print(features_2_train.shape)
print(features_2_valid.shape)
print(features_2_test.shape)

(127656, 132877)
(15957, 132877)
(15958, 132877)


#### Логистическая регрессия <a class="anchor" id="section_3_1"></a>

In [48]:
model_log_2 = LogisticRegression(random_state=12345, class_weight='balanced')

In [49]:
params_log_2 = {
    'solver': ['liblinear', 'sag','saga','newton-cg'],
    'C': [0.5,1.0,1.5],
    'intercept_scaling':[0.5,1.0,1.5]
}

In [50]:
model_log_2_grid = RandomizedSearchCV(model_log_2, params_log_2, scoring='f1')
model_log_2_grid.fit(features_2_train, target_2_train)

RandomizedSearchCV(estimator=LogisticRegression(class_weight='balanced',
                                                random_state=12345),
                   param_distributions={'C': [0.5, 1.0, 1.5],
                                        'intercept_scaling': [0.5, 1.0, 1.5],
                                        'solver': ['liblinear', 'sag', 'saga',
                                                   'newton-cg']},
                   scoring='f1')

In [51]:
print('Логистическая регрессия. Гиперпараметры ', model_log_2_grid.best_params_)
print('Логистическая регрессия. F1 ', model_log_2_grid.best_score_)

Логистическая регрессия. Гиперпараметры  {'solver': 'sag', 'intercept_scaling': 0.5, 'C': 1.5}
Логистическая регрессия. F1  0.7538282119093229


#### LightGBM <a class="anchor" id="section_3_2"></a>

In [52]:
model_lgb_2 = LGBMClassifier(random_state=12345, class_weight='balanced')

In [53]:
lgb_params_2 = {
    'objective': ['regression','binary'],
    'boosting_type' : ['dart','gbdt','goss'],
    'max_depth': [15,30],
    'num_leaves': [10,200,250]
}


In [54]:
model_lgb_2_grid = RandomizedSearchCV(model_lgb_2, lgb_params_2, scoring='f1')
model_lgb_2_grid.fit(features_2_train, target_2_train)

RandomizedSearchCV(estimator=LGBMClassifier(class_weight='balanced',
                                            random_state=12345),
                   param_distributions={'boosting_type': ['dart', 'gbdt',
                                                          'goss'],
                                        'max_depth': [15, 30],
                                        'num_leaves': [10, 200, 250],
                                        'objective': ['regression', 'binary']},
                   scoring='f1')

In [55]:
print('LightGBM. Гиперпараметры ', model_lgb_2_grid.best_params_)
print('LightGBM. F1 ', model_lgb_2_grid.best_score_)

LightGBM. Гиперпараметры  {'objective': 'binary', 'num_leaves': 200, 'max_depth': 30, 'boosting_type': 'goss'}
LightGBM. F1  0.7463515795992002


#### Случайный лес <a class="anchor" id="section_3_3"></a>

In [56]:
model_forest_2 = RandomForestClassifier(random_state=12345, class_weight='balanced')

In [57]:
forest_params_2 = {
    'max_depth': range(1, 26),
    'n_estimators':range(1, 11)
}

In [58]:
model_forest_2_grid = RandomizedSearchCV(model_forest_2, forest_params_2, scoring='f1')
model_forest_2_grid.fit(features_2_train, target_2_train)

RandomizedSearchCV(estimator=RandomForestClassifier(class_weight='balanced',
                                                    random_state=12345),
                   param_distributions={'max_depth': range(1, 26),
                                        'n_estimators': range(1, 11)},
                   scoring='f1')

In [59]:
print('Случайный лес. Гиперпараметры ', model_forest_2_grid.best_params_)
print('Случайный лес. F1 ', model_forest_2_grid.best_score_)

Случайный лес. Гиперпараметры  {'n_estimators': 9, 'max_depth': 17}
Случайный лес. F1  0.331443633785591


#### Вывод <a class="anchor" id="section_3_4"></a>
Из рассмотренных моделей: логистическая регрессия, LightGBM, случайный лес, только логистическая регрессия (гиперпараметры:  'solver': 'sag', 'intercept_scaling': 0.5, 'C': 1.5) позволила получить требуемое значение метрики качества F1 не меньше 0.75 - 0.753. Именно эту модель мы и протестируем.

### Тестирование лучшей модели<a class="anchor" id="chapter4"></a>

In [60]:
model_best = LogisticRegression(**model_log_2_grid.best_params_)
model_best.fit(features_2_train, target_2_train)
predictions_test = model_best.predict(features_2_test)
f1_Score = f1_score(target_2_test, predictions_test)
print('Логистическая регрессия. Тест. F1', f1_Score.round(2))

Логистическая регрессия. Тест. F1 0.74


Из рассмотренных моделей лучшее значение метрики качества F1 на обучающей выборке у логистической регресии (гиперпараметры:  'solver': 'sag', 'intercept_scaling': 0.5, 'C': 1.5). На тестовой чуть ниже - 0.74.

### Итоговый вывод<a class="anchor" id="chapter5"></a>

В полученной базе данных содержится 159571 записей. Текст комментариев - английский. Отрицательные комментарии составляют 10.2%.
В ходе работ были проанализированы 2 подхода - применение предобученной модели DistilBert для  токенизации текстов и создания эмбеддингов. Второй подход - применение метода TF-IDF.
В обоих случаях для обучения применялись модели: логистическая регрессия, LightGBM, случайный лес.
Лучший результат метрики качества F1 на обучающей выборке (0.753) получен при применении метода TF-IDF и логистической регрессии (гиперпараметры:  'solver': 'sag', 'intercept_scaling': 0.5, 'C': 1.5). На тестовой результат чуть хуже - F1 = 0.74.